## 9.10 Homework

In this homework, we'll deploy the dogs vs cats model we trained in the previous 
homework.

Download the model from here: 

https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats-model/dogs_cats_10_0.687.h5


In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install tensorflow

In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img

import tensorflow.lite as tflite

from io import BytesIO
from urllib import request

from PIL import Image

In [4]:
tf.__version__

'2.7.0'

In [5]:
!python -V

Python 3.7.4


In [6]:
model = keras.models.load_model('dogs_cats_10_0.687.h5')

## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the converted model? 

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('dogs_cats_model.tflite', 'wb') as f_out:
  f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/my/dfcsq7117_3_f5w7tt517zhw0000gp/T/tmpy7upz8n6/assets


In [8]:
!ls -lh

total 264240
-rw-r--r--@  1 lkirch  staff   104K Oct  7 17:24 20211007-Terminal Saved Output.txt
-rw-r--r--   1 lkirch  staff   138B Nov 28 09:08 Pipfile
-rw-rw-r--@  1 lkirch  staff   6.7K Nov 25 07:44 README.md
-rw-rw-r--@  1 lkirch  staff    27K Nov 25 07:44 appendix-b-python.ipynb
-rw-rw-r--@  1 lkirch  staff    49K Nov 25 07:44 appendix-c-numpy.ipynb
-rw-rw-r--@  1 lkirch  staff   141K Nov 25 07:44 appendix-d-pandas.ipynb
drwxrwxr-x@  4 lkirch  staff   128B Nov 25 07:44 chapter-02-car-price
drwxrwxr-x@  5 lkirch  staff   160B Nov 25 07:44 chapter-03-churn-prediction
drwxrwxr-x@ 13 lkirch  staff   416B Nov 25 07:44 chapter-05-deployment
drwxrwxr-x@  4 lkirch  staff   128B Nov 25 07:44 chapter-06-trees
drwxrwxr-x@  5 lkirch  staff   160B Nov 25 07:44 chapter-07-neural-nets
drwxrwxr-x@  8 lkirch  staff   256B Nov 25 07:44 chapter-08-serverless
drwxrwxr-x@  6 lkirch  staff   192B Nov 25 07:44 chapter-09-kubeflow
drwxrwxr-x@ 16 lkirch  staff   512B Nov 25 07:44 chapter-09-kubernetes
dr

## Answer 1

The size of the converted model is 43MB. 

## Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output. 

What's the output index for this model? 

In [9]:
interpreter = tflite.Interpreter(model_path='dogs_cats_model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [10]:
output_index

13

## Answer 2

The output index for this model is 13.

## Preparing the image

You'll need some code for downloading and resizing images. You can use 
this code:

```python
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
```

For that, you'll need to have pillow installed:

```bash
pip install pillow
```

Let's download and resize this image: 

https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg

Based on [the solution of the previous homework](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/08-deep-learning/CNN_solution.ipynb),
what should be the target size for the image? (150, 150)

In [11]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

In [12]:
def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [13]:
!pip install pillow

In [14]:
pug_img = download_image('https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg')

In [15]:
pug_img = prepare_image(pug_img, (150,150))

In [16]:
print(type(pug_img))

<class 'PIL.Image.Image'>


## Question 3

Now we need to turn the image into an numpy array and pre-process it. 

After the pre-processing, what's the value in the first pixel, the R channel?

In [26]:
x = np.array(pug_img)
X = np.array([x])
X.shape

(1, 150, 150, 3)

In [27]:
# rescale
X = np.float32(X*(1./255))
X[0][0][0]

array([0.7058824 , 0.77254903, 0.7490196 ], dtype=float32)

## Answer 3

After the pre-processing, the value in the first pixel, the R channel, is 0.7058824.

## Question 4

Now let's apply this model to this image. What's the output of the model?

In [28]:
preds = model.predict(X)

In [29]:
preds

array([[0.77049136]], dtype=float32)

## Answer 4

The output of the model is 0.77049136.  Indicating a dog.

## Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will 
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook 
by importing the file and invoking the function from this file.  


## Docker 

For the next two questions, we'll use a Docker image that I already 
prepared. This is the Dockerfile that I used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.8
COPY cats-dogs-v2.tflite .
```

And pushed it to [`agrigorev/zoomcamp-cats-dogs-lambda:v2`](https://hub.docker.com/r/agrigorev/zoomcamp-cats-dogs-lambda/tags).


> Note: The image already contains a model and it's not the same model
> as the one we used for questions 1-4.

## Question 5

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `cats-dogs-v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference).


What's the image id of the base image? 

In the build logs (on Linux), you'll see a log like that:

```
$ docker some-command-for-building
Sending build context to Docker daemon  2.048kB
Step 1/N : FROM agrigorev/zoomcamp-model:3.8.12-slim
 ---> XXXXXXXXXXXX
Step 2/N : ....
```

You need to get this `XXXXXXXXXXXX`. 

On MacOS and Windows, the logs for `docker build` are different. 
To get the image id there, you can use `docker image ls -a`.

## Answer 5

IMAGEID = 322fc756f258

## Question 6

Now run the container locally.

Score this image: https://upload.wikimedia.org/wikipedia/commons/1/18/Vombatus_ursinus_-Maria_Island_National_Park.jpg

What's the output from the model? 

## Answer 6
Running the model locally, I get 0.8457329273223877 for the adorable wombat picture.  This is closest to 0.74 in the answer selections.

## Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout 
* Test it
* Expose the lambda function using API Gateway

## Publishing to Docker hub

This is just for reference, this is how I published an image to Docker hub:

```bash
docker build -t cats-dogs-lambda .
docker tag cats-dogs-lambda:latest agrigorev/zoomcamp-cats-dogs-lambda:v2
docker push agrigorev/zoomcamp-cats-dogs-lambda:v2
```